In [13]:
# import locale
# locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
!pip3 install -q -U transformers
!pip3 install -q -U kaggle
!pip3 install -q -U accelerate
!pip3 install -q -U datasets


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cubinlinker, which is not installed.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires ptxcompiler, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.8 which is incompatible.
apache-beam 2.46.0 requires numpy<1.25.0,>=1.14.3, but you have numpy 1.26.4 which is incompatible.
apache-beam 2.46.0 requires pyarrow<10.0.0,>=3.0.0, but you have pyarrow 15.0.1 which is incompatible.
beatrix-jupyterlab 2023.128.151533 requires jupyterlab~=3.6.0, but you have jupyterlab 4.1.2 which is incompatible.
cudf 23.8.0 requires cuda-python<12.0a0,>=11.7.1, but you have cuda-python 12.3.0 which

In [6]:
from kaggle_secrets import UserSecretsClient
secret_label = "HUGGINGFACE_TOKEN"
secret_value = UserSecretsClient().get_secret(secret_label)
from huggingface_hub import login
login(token=secret_value)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [7]:
import torch
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM,GenerationConfig

In [32]:
model_id = "merged_model"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda:0")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [33]:
def formatting_prompts_func(example):

    return  f"### summarize the following data : {example}\n ### Answer: "


In [34]:
from datasets import load_dataset,Dataset

dataset = load_dataset("zqz979/meta-review", split="test")

In [41]:
def extract_answer(llm_output):
    parts = llm_output.split("### Answer:")
    if len(parts) > 1:
        return parts[1].strip()  # Extract the answer part
    else:
        return None  # No '### Answer:' found 


In [42]:
device = "cuda:0"
encodeds = tokenizer(formatting_prompts_func(dataset[2]["Input"]), return_tensors="pt")


model_inputs = encodeds.to(device)

generated_ids = model.generate(**model_inputs,max_new_tokens=200,repetition_penalty=10.0)
decoded = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
print(extract_answer(decoded))

Both `max_new_tokens` (=200) and `max_length`(=5000) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


1st version accepted after minor revisions    I am happy about accepting your submission because you provided detailed answers addressing my questions during rebuttal period! Thank You :)     ### Summary Overall summary score = accept [+]      This manuscript presents intrinsic dimension & cluster learn ability measures based upon similarity measure derived via knn classifier predictions across multiple classes within one image set / domain . They propose deepcluster modification incorporating those features alongside traditional align unif predictor resulting improved predictive power overall vs prior art approaches like aligned unifier etc.. Additionally several ablation studies were performed showing benefits associated wrt adding them back together again at inference time leading up towards future research directions around improving robustness against adversarial attacks through leveraging similar ideas presented here ! My major concern was whether aligning images first might help

In [36]:
dataset

Dataset({
    features: ['Input', 'Output'],
    num_rows: 1649
})

In [37]:
original_model_id = "google/gemma-2b"
original_tokenizer = AutoTokenizer.from_pretrained(original_model_id)
original_model = AutoModelForCausalLM.from_pretrained(original_model_id, device_map="cuda:1")

tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/555 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/627 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/13.5k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/67.1M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [44]:
input_ids = tokenizer(dataset[0]["Input"], return_tensors="pt").input_ids.to("cuda:1")
original_model_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=300),repetition_penalty=10.0)
original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

In [47]:
raw_input = dataset[0:10]['Input']
human_baseline_summaries = dataset[0:10]['Output']

original_model_summaries = []

peft_model_summaries = []

for i in range(10):
    prompt = f"### summarize the following data : {raw_input[i]}\n ### Answer: "

    input_ids1 = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:0")
    input_ids2 = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda:1")
    human_baseline_text_output = human_baseline_summaries[i]

    original_model_outputs = original_model.generate(input_ids=input_ids2, generation_config=GenerationConfig(max_new_tokens=300),repetition_penalty=10.0)
    original_model_text_output = original_tokenizer.decode(original_model_outputs[0], skip_special_tokens=True)

    peft_model_outputs = model.generate(input_ids=input_ids1, generation_config=GenerationConfig(max_new_tokens=300),repetition_penalty=10.0)
    peft_model_text_output = extract_answer(tokenizer.decode(peft_model_outputs[0], skip_special_tokens=True))

    original_model_summaries.append(original_model_text_output)

    peft_model_summaries.append(peft_model_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns = ['human_baseline_summaries', 'original_model_summaries', 'peft_model_summaries'])
df

,human_baseline_summaries,original_model_summaries,peft_model_summaries
0,The paper present results using syntactic info...,### summarize the following data : This paper...,1 - Minor typos throughout manuscript 2 – O...
1,The authors focus on large scale out-of-distri...,### summarize the following data : The authors...,1st version accepted after author response #1 ...
2,The problem studied in this paper is interesti...,### summarize the following data : This paper ...,1st version accepted after minor revisions ...
3,The major criticism of this paper after the in...,### summarize the following data : This paper ...,1 - Minor typos throughout; no major issues fo...
4,This paper proposes a novel challenge setting:...,### summarize the following data : This work p...,1st version accepted after minor revisions bas...
5,The paper explores generation of a volumetric ...,### summarize the following data : The work ta...,1) Overall Score - Accept after minor revision...
6,The paper introduces the new task of few-shot ...,### summarize the following data : A. Summary...,1.) Overall Score = Accept after Minor Revisio...
7,This paper studies the computational benefits ...,### summarize the following data : The paper c...,5 out if possible score because while some par...
8,The proposed approach exploits the color distr...,### summarize the following data : This paper ...,9/17 - Accepted as submitted after minor revis...
9,The paper proposes a variant of recurrent neur...,### summarize the following data : In this pap...,1st version accepted! ## Strengths * Interesti...


In [52]:
!pip3 install -q -U rouge_score
!pip3 install -q -U evaluate
import evaluate
rouge = evaluate.load('rouge')


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2024-03-12 05:47:28.041225: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-12 05:47:28.041357: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to re

In [54]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries[0:len(original_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

peft_model_summaries = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True,
)

print('ORIGINAL MODEL:')
print(original_model_results)
print('PEFT MODEL:')
print(peft_model_summaries)

ORIGINAL MODEL:
{'rouge1': 0.16258186024473512, 'rouge2': 0.07159352283778986, 'rougeL': 0.09863675265565547, 'rougeLsum': 0.10117953633846334}
PEFT MODEL:
{'rouge1': 0.10872956206056991, 'rouge2': 0.006127211290614896, 'rougeL': 0.04466714404696985, 'rougeLsum': 0.04479423325928805}
